In [11]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
import os
import pandas as pd
from random import randint
from bs4 import BeautifulSoup as bs
import requests
from tqdm import tqdm
from fuzzywuzzy import process

In [3]:
masters_results = pd.DataFrame(columns = ['year', 'finish_position', 'player_name', 'to_par_score', 
            'round_1_score', 'round_2_score', 'round_3_score', 'round_4_score', 'total_score',
            'earnings'])

player_scorecards = pd.DataFrame(columns = ['year', 'player', 'round_num', 'hole_1', 'hole_2', 'hole_3', 'hole_4',
        'hole_5', 'hole_6', 'hole_7', 'hole_8', 'hole_9', 'front_total','hole_10', 'hole_11', 'hole_12', 'hole_13',
        'hole_14', 'hole_15', 'hole_16', 'hole_17', 'hole_18', 'back_total', 'total_score'])


In [4]:
row_count = 0
score_count = 0

for year in tqdm(range(2016, 1936, -1)):
    url = 'http://www.augusta.com/masters/historic/leaderboards/' + str(year) + 'leaderboard.shtml'
    
    time.sleep(1)
    response = requests.get(url)
    soup = bs(response.content, 'lxml')
    
    tr_ls = soup.findAll('table')[0].findAll('tr')

   
    for t, tr in enumerate(tr_ls[1:]):
        td = tr.findAll('td')
        cell_ls = [year]
        for c, cell in enumerate(td):
            cell_ls.append(cell.text.replace('\n', '').replace('\t',''))
        masters_results.loc[row_count] = cell_ls
        row_count += 1
    
    a_tags = soup.find_all('a')

    player_urls = []
    for a_tag in a_tags:
        if 'historic/players/hbh' in a_tag.attrs['href']:
            player_urls.append(a_tag)
            
    for player in player_urls:
        player_name = player.text
        player_url = 'http://www.augusta.com' + player.attrs['href']
        
        time.sleep(1)
        player_response = requests.get(player_url)

        player_soup = bs(player_response.content, 'lxml')
        
        player_tables = player_soup.findAll('table')

        for t, tr in enumerate(player_tables[0].findAll('tr', {'id': ''})):
            td = tr.findAll('td')
            td_0 = td[0].text.replace('\n','').replace('\t','')
            if td_0[:2] == 'Rd':
                rd = td_0[-1]
                cell_ls = [year, player_name, rd]
                for cell in td[1:]:
                    temp_cell = cell.text.replace('\n','').replace('\t','')
                    cell_ls.append(temp_cell)
                player_scorecards.loc[score_count] = cell_ls
                score_count += 1
                

 89%|████████▉ | 71/80 [1:42:27<14:01, 93.51s/it] 

IndexError: list index out of range

In [34]:
os.chdir('/home/valesco/Datasets/PGA_Data/pga_master_files/')
player_scorecards.to_csv('masters_player_scorecards.csv', index = False, sep = ';')
masters_results.to_csv('masters_results.csv', index = False, sep = ';')

In [17]:
a_tags = soup.find_all('a')

player_urls = []
for a_tag in a_tags:
    if 'historic/players/hbh' in a_tag.attrs['href']:
        player_urls.append(a_tag)
        
player_urls

[<a href="/masters/historic/players/hbh/2007_hbh11981.shtml">Zach Johnson</a>,
 <a href="/masters/historic/players/hbh/2007_hbh209.shtml">Retief Goosen</a>,
 <a href="/masters/historic/players/hbh/2007_hbh1191.shtml">Rory Sabbatini</a>,
 <a href="/masters/historic/players/hbh/2007_hbh184.shtml">Tiger Woods</a>,
 <a href="/masters/historic/players/hbh/2007_hbh1159.shtml">Jerry Kelly</a>,
 <a href="/masters/historic/players/hbh/2007_hbh3185.shtml">Justin Rose</a>,
 <a href="/masters/historic/players/hbh/2007_hbh197.shtml">Stuart Appleby</a>,
 <a href="/masters/historic/players/hbh/2007_hbh1508.shtml">Padraig Harrington</a>,
 <a href="/masters/historic/players/hbh/2007_hbh220.shtml">David Toms</a>,
 <a href="/masters/historic/players/hbh/2007_hbh6962.shtml">Paul Casey</a>,
 <a href="/masters/historic/players/hbh/2007_hbh3128.shtml">Luke Donald</a>,
 <a href="/masters/historic/players/hbh/2007_hbh4483.shtml">Vaughn Taylor</a>,
 <a href="/masters/historic/players/hbh/2007_hbh205.shtml">Tim 

In [66]:
tr_ls = soup.findAll('table')[0].findAll('tr')

row_count = 0
for t, tr in enumerate(tr_ls[1:]):
    td = tr.findAll('td')
    cell_ls = [year]
    for c, cell in enumerate(td):
        cell_ls.append(cell.text.replace('\n', '').replace('\t',''))
    masters_results.loc[row_count] = cell_ls
    row_count += 1
    
masters_results

,year,finish_position,player_name,to_par_score,round_1_score,round_2_score,round_3_score,round_4_score,total_score,earnings
0,2007,1,Zach Johnson,+1,71,73,76,69,289,"$1,305,000"
1,2007,T2,Retief Goosen,+3,76,76,70,69,291,"$541,333"
2,2007,T2,Rory Sabbatini,+3,73,76,73,69,291,"$541,333"
3,2007,T2,Tiger Woods,+3,73,74,72,72,291,"$541,333"
4,2007,T5,Jerry Kelly,+4,75,69,78,70,292,"$275,500"
5,2007,T5,Justin Rose,+4,69,75,75,73,292,"$275,500"
6,2007,T7,Stuart Appleby,+5,75,70,73,75,293,"$233,812"
7,2007,T7,Padraig Harrington,+5,77,68,75,73,293,"$233,812"
8,2007,9,David Toms,+6,70,78,74,72,294,"$210,250"
9,2007,T10,Paul Casey,+7,79,68,77,71,295,"$181,250"


In [79]:
player_url = 'http://www.augusta.com' + player_urls[3].attrs['href']
player_response = requests.get(player_url)

player_soup = bs(player_response.content, 'lxml')

In [81]:
player_tables = player_soup.findAll('table')

#score_count = 0
for t, tr in enumerate(player_tables[0].findAll('tr', {'id': ''})):
    td = tr.findAll('td')
    td_0 = td[0].text.replace('\n','').replace('\t','')
    if td_0[:2] == 'Rd':
        rd = td_0[-1]
        cell_ls = [year, player_name, rd]
        for cell in td[1:]:
            temp_cell = cell.text.replace('\n','').replace('\t','')
            cell_ls.append(temp_cell)
        player_scorecards.loc[score_count] = cell_ls
        score_count += 1
            

In [82]:
player_scorecards

,year,player,round,hole_1,hole_2,hole_3,hole_4,hole_5,hole_6,hole_7,...,hole_11,hole_12,hole_13,hole_14,hole_15,hole_16,hole_17,hole_18,back_total,total_score
0,2007,Zach Johnson,4,5,4,3,3,5,3,4,...,4,3,4,3,5,2,5,4,34,69
1,2007,Zach Johnson,3,4,5,4,3,5,3,4,...,5,4,5,4,4,4,4,4,38,76
2,2007,Zach Johnson,2,4,4,3,3,5,3,4,...,5,3,4,4,5,4,5,5,39,73
3,2007,Zach Johnson,1,5,4,4,3,4,3,4,...,4,3,4,4,4,4,4,4,36,71
4,2007,Tiger Woods,4,5,4,4,3,4,4,4,...,4,3,3,4,5,3,4,4,35,72
5,2007,Tiger Woods,3,4,5,3,3,4,3,4,...,4,4,4,4,5,3,5,5,38,72
6,2007,Tiger Woods,2,5,5,4,4,4,3,5,...,5,4,5,4,4,3,3,4,35,74
7,2007,Tiger Woods,1,4,5,4,3,4,3,5,...,4,3,4,4,4,3,5,5,36,73


In [12]:
player_ids = pd.read_csv('/home/valesco/Datasets/PGA_Data/player_ids.csv', sep = ';')
player_names_ls = player_ids['player_name'].values
player_ids.head()

,player_id,player_name,first_name,last_name
0,95034,Merrick Francis,Merrick,Francis
1,33952,Erick Morales,Erick,Morales
2,6631,Todd McCorkle,Todd,McCorkle
3,22621,Ben Curtis,Ben,Curtis
4,47992,Matt Mabrey,Matt,Mabrey


In [17]:
def return_pid(x):
    matched_name = process.extractOne(x, player_names_ls)[0]
    return player_ids['player_id'].loc[player_ids['player_name'] == matched_name].values[0]

#masters_results['player_id'] = masters_results['player_name'].apply(lambda x: return_pid(x))
player_scorecards['player_id'] = player_scorecards['player'].apply(lambda x: return_pid(x))

masters_results.head()

,year,finish_position,player_name,to_par_score,round_1_score,round_2_score,round_3_score,round_4_score,total_score,earnings,player_id
0,2016.0,1,Danny Willett,-5,70,74,72,67,283,"$1,800,000",32139
1,2016.0,T2,Jordan Spieth,-2,66,74,73,73,286,"$880,000",34046
2,2016.0,T2,Lee Westwood,-2,71,75,71,69,286,"$880,000",20396
3,2016.0,T4,Paul Casey,-1,69,77,74,67,287,"$413,333",25364
4,2016.0,T4,J.B. Holmes,-1,72,73,74,68,287,"$413,333",27141


In [29]:
masters_results['to_par_score'] = masters_results['to_par_score'].apply(lambda x: 
        x.replace('E', '0'))
#masters_results['earnings'] = masters_results['earnings'].apply(lambda x: x.replace('$', '').replace(',',''))

masters_results.head()

,player_id,player_name,year,finish_position,to_par_score,round_1_score,round_2_score,round_3_score,round_4_score,total_score,earnings
0,32139,Danny Willett,2016.0,1,-5,70,74,72,67,283,1800000
1,34046,Jordan Spieth,2016.0,2,-2,66,74,73,73,286,880000
2,20396,Lee Westwood,2016.0,2,-2,71,75,71,69,286,880000
3,25364,Paul Casey,2016.0,4,-1,69,77,74,67,287,413333
4,27141,J.B. Holmes,2016.0,4,-1,72,73,74,68,287,413333


In [64]:
masters_results = masters_results[['player_id', 'player_name', 'year', 'finish_position', 'to_par_score',
                'round_1_score', 'round_2_score', 'round_3_score', 'round_4_score', 'total_score', 'earnings']]

In [33]:
col_ls = ['player_id', 'player', 'year', 'round_num', 'hole_1', 'hole_2', 'hole_3', 'hole_4',
       'hole_5', 'hole_6', 'hole_7', 'hole_8', 'hole_9', 'front_total',
       'hole_10', 'hole_11', 'hole_12', 'hole_13', 'hole_14', 'hole_15',
       'hole_16', 'hole_17', 'hole_18', 'back_total', 'total_score']

player_scorecards = player_scorecards[col_ls]